In [1]:
from fake_useragent import UserAgent
from parsel import Selector
import requests
import time
from bs4 import BeautifulSoup
import json
import os
import re
import pandas as pd
import numpy as np


In [35]:
# Ignorar FutureWarnings
import warnings
import time
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

Função: get_user_agent

Propósito:

Gerar um cabeçalho HTTP com um agente de usuário (User-Agent) aleatório.
Descrição:

A função cria uma instância da classe UserAgent.
Obtém um User-Agent aleatório usando o método random.
Retorna um dicionário contendo o User-Agent gerado.
Aplicação:

Utilizada para fazer requisições HTTP com User-Agents diferentes, ajudando a evitar bloqueios por servidores que detectam acessos automatizados.

In [ ]:
def get_user_agent():
    ua = UserAgent()
    header = {'User-Agent':str(ua.random)}

    return header

Função: verifica_e_le_json


Propósito:

Verificar a existência de um arquivo JSON e ler seu conteúdo, caso o arquivo exista. Salvamos a informação com o link do campeonato e os links dos times em um 
JSON local, para otimizar o processo
    
Descrição:

Verificação de Existência: A função verifica se o arquivo especificado pelo caminho existe.
Leitura do Arquivo: Se o arquivo existir, ele é aberto em modo de leitura com codificação UTF-8.
Carregamento do JSON: O conteúdo do arquivo é carregado em um dicionário usando a biblioteca json.
Retorno dos Dados: Se o arquivo existe, os dados são retornados; caso contrário, a função retorna None.

    
Aplicação:

Útil para garantir que um arquivo JSON exista antes de tentar lê-lo, evitando erros de arquivo não encontrado e facilitando o manuseio seguro de dados JSON em aplicações.

In [2]:
def verifica_e_le_json(caminho_arquivo):
    if os.path.exists(caminho_arquivo):
        with open(caminho_arquivo, 'r', encoding='utf-8') as arquivo:
            dados = json.load(arquivo)
        return dados
    else:
        return None


Função: extract_number

Propósito:

Extrair o primeiro número de uma string que está dividida por barras ("/"). 


Descrição:

Divisão da String: A função divide a string de entrada pelos caracteres de barra ("/"), criando uma lista de partes.
Busca por Números: Percorre cada parte da lista resultante.
Verificação de Dígitos: Verifica se cada parte é composta apenas por dígitos.
Retorno do Número: Retorna a primeira parte que contém apenas dígitos.

Aplicação:

Cada competição possui um código que fica no seu link (exemplo: https://fbref.com/en/comps/9/Premier-League-Stats  - O código da Premier League é 9). Usamos essa função para extrair esse código do link

In [3]:
def extract_number(text):
    # Divide a string pelos caracteres de barra ("/")
    text_parts = text.split('/')

    # Procura o número na lista resultante
    for part in text_parts:
        if part.isdigit():
            return part

Função: adjust_column


Propósito:

Ajustar os nomes das colunas de um DataFrame para simplificar e limpar os rótulos.
Descrição:

Combinação de Nomes de Colunas: Combina múltiplas camadas de nomes de colunas em uma única string para cada coluna, removendo espaços desnecessários.
Inicialização da Lista de Novas Colunas: Cria uma lista vazia para armazenar os novos nomes de colunas.
Iteração sobre as Colunas:
Para cada nome de coluna combinado, verifica se contém a substring 'level_0'.
Substituição Condicional:
Se a coluna contém 'level_0', pega a última palavra do nome da coluna.
Caso contrário, mantém o nome original da coluna.
Armazenamento dos Novos Nomes: Adiciona cada novo nome de coluna à lista de novas colunas.
Retorno das Novas Colunas: Retorna a lista com os nomes das colunas ajustados.
Aplicação:

Útil para limpar e simplificar os nomes de colunas de um DataFrame. As colunas das tabelas vem desformatadas, pelo fato de muitas delas estarem agrupadas em mais de uma célula. Aqui, fazemos o ajuste do nome das colunas.

In [4]:
def adjust_column(df):
    df_columns = [' '.join(col).strip() for col in df.columns]
    new_columns = []

    for col in df_columns:
        if 'level_0' in col:
            new_col = col.split()[-1]  # takes the last name
        else:
            new_col = col
        new_columns.append(new_col)
    return new_columns     

Função: get_competition_coverage_all


Propósito:

Obter e salvar informações de cobertura de competições esportivas a partir do link do FBREF

Descrição:

Requisição da Página: Acessa a página de cobertura de competições do site FBref.

Análise da Página: Utiliza BeautifulSoup para analisar o conteúdo HTML da página.

Inicialização da Lista: Cria uma lista vazia para armazenar os dados das competições.

Iteração sobre as Competições: Percorre todos os elementos <td> com o atributo data-stat='comp'.

Extração de Informações: Para cada competição, extrai o nome, link, país, gênero, temporadas, e os anos de início e fim da cobertura.

Formação do Link de Estatísticas: Gera o link para as estatísticas de cada competição.

Criação do Dicionário: Cria um dicionário com as informações extraídas e calculadas.

Armazenamento na Lista: Adiciona o dicionário à lista de competições.

Salvamento dos Dados: Converte a lista de competições em JSON e salva em um arquivo.

Retorno das Competições: Retorna a lista de competições com todas as informações coletadas.

    
Aplicação:

Esta função é útil para coletar, organizar e salvar dados de cobertura de competições esportivas do FBREF em um formato estruturado, facilitando análises e consultas posteriores.






In [5]:
def get_competition_coverage_all():

    access_link = 'https://fbref.com/en/about/coverage'
    data = requests.get(access_link,headers=get_user_agent())
    soup = BeautifulSoup(data.text, features="lxml")

    competitions = []

    for comp in soup.find_all('td', {'data-stat': 'comp'}):
        comp_link = comp.find('a')
        comp_text = comp_link.text.strip() if comp_link else ""
        comp_href = comp_link.get('href') if comp_link else ""
        maxseason =  comp.find_next_sibling('td', {'data-stat': 'maxseason'}).text.strip()
        stats_link = r'https://fbref.com/en/comps/' + extract_number(comp_href) + '/' + maxseason + '/' + maxseason + '-' + comp_text.replace(' ','-') + '-Stats'
            
        competition_data = {
            "comp": comp_text,
            "country": comp.find_next_sibling('td', {'data-stat': 'country'}).text.strip(),
            "hist_link": r'https://fbref.com' + comp_href,
            "stats_link":stats_link,
            "gender": comp.find_next_sibling('td', {'data-stat': 'gender'}).text.strip(),
            "seasons": comp.find_next_sibling('td', {'data-stat': 'Seasons'}).text.strip(),
            "minseason": comp.find_next_sibling('td', {'data-stat': 'minseason'}).text.strip(),
            "maxseason": maxseason
        }
            
        competitions.append(competition_data)

        #json_data = json.dumps(competitions, indent=4)

    with open('competitions_available.json', 'w') as json_file:
        json.dump(competitions, json_file)

    return competitions

In [6]:
    def return_team_links(gender='All'):

  
        json_data = verifica_e_le_json('C:\\Users\\dpsoe\\Downloads\\competitions_available.json') #get_competition_coverage_all()

        if json_data is None:
            json_data = get_competition_coverage_all()

        link_teams = {}
        num_req = 0

        if gender == 'All':
            list_gender = ['M','F']
        else:
            list_gender = list(gender)

        
        for data in json_data:

            
            if "2024" not in data["maxseason"]:
                continue

            if data["gender"] not in list_gender:
                continue

            if data["country"] == '':
                continue
            
            link = data["stats_link"]
            comp_name = data["comp"]
            gender = data["gender"]
            season = data["maxseason"]
            
            if num_req <= 20:
              data = requests.get(link, headers=get_user_agent())
              num_req += 1
            else:
              time.sleep(30)
              data = requests.get(link, headers=get_user_agent())
              num_req = 1
            
            soup =  BeautifulSoup(data.text,features="lxml")
            all_links = soup.find_all('td', {'data-stat': 'team'})
            
            list_links_teams = set()
            for links in all_links:
              all = links.find_all('a')
              for link in all:
                href = link.get('href')
                if href:
                  href_final = f'https://fbref.com' + href
                  list_links_teams.add(href_final)

                #Retorna os links dos times
            if gender not in link_teams.keys():
                link_teams[gender] = {}

            if comp_name not in link_teams[gender].keys():
                link_teams[gender][comp_name] = {}

            if season not in link_teams[gender][comp_name] .keys():
                link_teams[gender][comp_name][season] = list(list_links_teams)


        with open('link_teams_available.json', 'w') as json_file:
          json.dump(link_teams, json_file)

        return link_teams



In [20]:
teste = return_team_links(gender='All')

In [21]:
teste

{'M': {'FA Cup': {'2023-2024': []},
  'FA Community Shield': {'2023-2024': []},
  'EFL Cup': {'2023-2024': []},
  'Copa del Rey': {'2023-2024': []},
  'Supercopa de España': {'2023-2024': []},
  'Coupe de France': {'2023-2024': []},
  'Trophée des Champions': {'2023-2024': []},
  'DFB-Pokal': {'2023-2024': []},
  'DFL-Supercup': {'2023-2024': []},
  'Coppa Italia': {'2023-2024': []},
  'Supercoppa Italiana': {'2023-2024': []},
  'Copa de la Liga Profesional': {'2024': ['https://fbref.com/en/squads/41c139b6/Velez-Sarsfield-Stats',
    'https://fbref.com/en/squads/a4570206/Defensa-y-Justicia-Stats',
    'https://fbref.com/en/squads/e2f19203/Instituto-Stats',
    'https://fbref.com/en/squads/950a95f2/Gimnasia-y-Esgrima-LP-Stats',
    'https://fbref.com/en/squads/9bf4eaf4/Newells-Old-Boys-Stats',
    'https://fbref.com/en/squads/7765008b/Belgrano-Stats',
    'https://fbref.com/en/squads/0e92bf17/Tigre-Stats',
    'https://fbref.com/en/squads/b3d222b1/Deportivo-Riestra-Stats',
    'https://

In [61]:
   def get_stats_team_table(data,list_not_have,club_name,competition,gk_avncd_info = False,only_std_stats = True):

        ctrl_list = []
        list_std_stats = []
        dict_stats = {}

        stats_list =  ['Standard Stats', 'Miscellaneous Stats', 'Goalkeeping','Playing Time','Possession','Defensive Actions','Advanced Goalkeeping','Shooting','Passing','Pass Types','Goal and Shot Creation']

        if gk_avncd_info == False:
            stats_list.remove('Advanced Goalkeeping')

        dict_stats[club_name] = {}

        try:
            df_std_stats = pd.read_html(data.text, match='Standard Stats')[0]
            df_std_stats.columns = adjust_column(df_std_stats)
            df_std_stats['Club'] = club_name
            df_std_stats['Competition'] = competition

            list_std_stats.append(df_std_stats)
            dict_stats[club_name]['Standard Stats'] = df_std_stats.to_json(orient='records') 

            
            ctrl_list.append('Standard Stats')
        except ValueError as e:
            dict_stats = {}

        
        if only_std_stats == True:
            return dict_stats
        else:
            list_stats = []

            if len(list_not_have) == 0:
                stats_dont_have = set()
            else:
                stats_dont_have = list_not_have

            for stats in stats_list:
                if (stats not in ctrl_list) and (stats not in stats_dont_have) :
                    try:
                        df = pd.read_html(data.text, match=stats)[0]
                        df.columns = adjust_column(df)
                  #      list_stats.append(df)
                        dict_stats[club_name][stats] = df.to_json(orient='records') 

                        ctrl_list.append(stats)
                        #df.columns = list(df.columns)
                    except ValueError as e:
                        stats_dont_have.add(stats)
                        continue
                else:
                    continue
            
            return dict_stats,stats_dont_have

In [62]:
    def main_get_stats_clubs(gender='All'):

        if gender != 'All':
          json_file = verifica_e_le_json('link_teams_available.json')[gender] #get_competition_coverage_all()
          links_teams_json = {}
          links_teams_json[gender] = json_file
        else:
          links_teams_json = verifica_e_le_json('link_teams_available.json')

        if links_teams_json is None:
            links_teams_json = return_team_links(gender)


        req_num = 0
        final_dict = {}
        clubs_info_list = []
        dont_have_dict = {}


        for gender in links_teams_json.keys():
            final_dict[gender] = {}
            print(gender)
            for comp in links_teams_json[gender].keys():
                comp_name = comp
                dont_have_dict[comp_name] = set()

                final_dict[gender][comp_name] = {}
                for season in links_teams_json[gender][comp_name].keys():
                    current_season = season
 

                    list_links_team = links_teams_json[gender][comp_name][current_season]

                    if len(list_links_team) == 0:
                        continue
                    print(comp_name)

                    for link in list_links_team:

                        list_not_have = dont_have_dict[comp_name]

                        match = re.search(r'/([^/]+)-Stats$', link)
                        ext_texto = match.group(1)
                        nome_clube = ext_texto.replace('-',' ')

                        if req_num <= 20:

                            data = requests.get(link, headers=get_user_agent())
                            req_num += 1

                            if data.status_code in (500,403,404):
                                continue

                            dict_stats,dont_have_list = get_stats_team_table(data,list_not_have,club_name = nome_clube,competition = comp_name,gk_avncd_info = False,only_std_stats = False)
                            dont_have_dict[comp_name] = dont_have_list
                            clubs_info_list.append(dict_stats)
                        
                        else:
                            time.sleep(60)
                            req_num = 0

                            data = requests.get(link, headers=get_user_agent())
                            req_num += 1

                            if data.status_code in (500,403,404):
                                continue

                            dict_stats,dont_have_list = get_stats_team_table(data,list_not_have,club_name = nome_clube,competition = comp_name,gk_avncd_info = False,only_std_stats = False)
                            dont_have_dict[comp_name] = dont_have_list
                            clubs_info_list.append(dict_stats)

                    final_dict[gender][comp_name][season] = clubs_info_list



       # with open('all_stats.json', 'w') as json_file:
      #    json.dump(final_dict, json_file)
            
        return final_dict

In [63]:
final_dict = main_get_stats_clubs(gender='F')

F
FA Women's Super League
Liga F
Championnat de France de Football Féminin
Frauen-Bundesliga
Serie A
A-League Women
ÖFB Frauen-Bundesliga
Belgian Women's Super League
Brasileirão Feminino Série A1
Danish Women's League
Women Empowerment League
Eredivisie Vrouwen
Toppserien
Swiss Women's Super League
Damallsvenskan
National Women's Soccer League
